# Characterizing Datasets

TJ Kim <br/>
3/14/21

Updated <br/>
3/14/21

#### Objective: 
Characterize dataset that are not i.i.d distributed: <br/>

Deliverables
- Guassian extrapolation of mean and variance of each "feature node"
- PAC representation of the point clusters 

The different clusters will be divided along:
- nth data set for each client
- across class lines
- combination of 2 prior


In [2]:
cd '/home/ubuntu/FedAtk/' 

/home/ubuntu/FedAtk


### Load Relevant Libraries and Modules

Load the relevant libraries for the federated learning code.

In [3]:
# Transferer
from transfer_attacks.Transferer import *
from configs.overwrite_config import *

# General Libraries
import torch
import numpy as np
import os
import pandas as pd